[Automated Options Trading Bot in Python](https://youtu.be/eBGcv97WXNA?si=lpj_USWhO8nMA9cT)



In [ ]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from thetadata import ThetaClient, OptionReqType, OptionRight, DateRange

def create_signals(ticker, exp_date, client):
    transactions = {
        "transaction_date": [],
        "ticker": [],
        "strike": [],
        "exp_date": [],
        "transaction_type": [],
    }

    strikes = client.get_strikes(ticker, exp_date)
    for strike in strikes:
        try:
            data = client.get_hist_option(
                req=OptionReqType.EOD,
                root=ticker,
                exp=exp_date,
                strike=strike,
                right=OptionRight.CALL, 
                date_range=DateRange(exp_date - dt.timedelta(days=90), exp_date)
                # date_range=DateRange(exp_date - dt.timedelta(days=7), exp_date)
            )
            print(data)

            # Signaling Logic
            if len(data) > 10:
                data.columns = ["Open", "High", "Low", "Close", "Volume", "Date"]
                data.set_index("Date", inplace=True)
                data["Signal"] = data["Volume"] > data["Volume"].mean() + 3 * data["Volume"].std()
                selected_data = data[data["Signal"]]

            
                for index, row in selected_data.iterrows():
                    transactions["transaction_date"].append(index)
                    transactions["ticker"].append(ticker)
                    transactions["strike"].append(strike)
                    transactions["exp_date"].append(exp_date)
                    transactions["transaction_type"].append("buy")

        except:
            continue

        try:
            data = client.get_hist_option(
                req=OptionReqType.EOD,
                root=ticker,
                exp=exp_date,
                strike=strike,
                right=OptionRight.PUT, 
                date_range=DateRange(exp_date - dt.timedelta(days=90), exp_date)
                # date_range=DateRange(exp_date - dt.timedelta(days=7), exp_date)
            )
            print(data)

            # Signaling Logic
            if len(data) > 10:
                data.columns = ["Open", "High", "Low", "Close", "Volume", "Date"]
                data.set_index("Date", inplace=True)
                data["Signal"] = data["Volume"] > data["Volume"].mean() + 3 * data["Volume"].std()
                selected_data = data[data["Signal"]]

            
                for index, row in selected_data.iterrows():
                    transactions["transaction_date"].append(index)
                    transactions["ticker"].append(ticker)
                    transactions["strike"].append(strike)
                    transactions["exp_date"].append(exp_date)
                    transactions["transaction_type"].append("sell")

        except:
            continue
        
    pd.DataFrame(transactions).to_csv(f"transactions/{ticker}_{exp_date}.csv")

def backtest(ticker, exp_date, client):
    df = pd.read_csv(f"transactions/{ticker}_{exp_date}.csv")
    df = df.sort_values(by="transaction_date")
    df.set_index("transaction_date", inplace=True)
    df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
    
    strikes = sorted(set(df.strike.values))
    total_profit = 0
    for strike in strikes:
        data = client.get_hist_option(
            req=OptionReqType.EOD,
            root=ticker,
            exp=exp_date,
            strike=strike,
            right=OptionRight.CALL, 
            date_range=DateRange(exp_date - dt.timedelta(days=90), exp_date)
        )

        data.columns = ["Open", "High", "Low", "Close", "Volume", "Date"]
        data.set_index("Date", inplace=True)

        plt.plot(data.index, data.Close)
        buy_data = df[df.transaction_type == "buy"]
        buy_data = buy_data[buy_data.strike == strike]
        filtered_data = data[data.index.isin(buy_data.index)]
        plt.scatter(filtered_data.index, filtered_data.Close, marker="^", color="green")

        plt.plot(data.index, data.Close)
        sell_data = df[df.transaction_type == "sell"]
        sell_data = sell_data[sell_data.strike == strike]
        filtered_data = data[data.index.isin(sell_data.index)]
        plt.scatter(filtered_data.index, filtered_data.Close, marker="v", color="red")
        plt.show()

client = ThetaClient(username=open("email.txt", "r").read(), passwd=open("passwd.txt", "r").read())

with client.connect():
    ticker = "BMY"
    exp_dates = client.get_expirations(ticker)

    for exp_date in exp_dates[390:400]:
        try:
            create_signals(ticker, exp_date, client)
            backtest(ticker, exp_date, client)
        except Exception as e:
            continue